In [1]:
import dlib
import cv2
import os
import re
import json
import numpy as np
import pandas as pd
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
import face_recognition

In [2]:
train_sample_metadata = pd.read_json('../input/deepfake-detection-challenge/train_sample_videos/metadata.json').T
train_sample_metadata

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4
...,...,...,...
etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4
etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4
eudeqjhdfd.mp4,REAL,train,None


In [3]:
train_sample = '../input/deepfake-detection-challenge/train_sample_videos/'
train_video_files = [train_sample + x for x in sorted(os.listdir(train_sample))]

In [4]:
train_video_files.pop()

'../input/deepfake-detection-challenge/train_sample_videos/metadata.json'

In [5]:
len(train_video_files)

400

In [6]:
sorted(train_video_files)

['../input/deepfake-detection-challenge/train_sample_videos/aagfhgtpmv.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/aapnvogymq.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/abarnvbtwb.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/abofeumbvv.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/abqwwspghj.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/acifjvzvpm.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/acqfdwsrhi.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/acxnxvbsxk.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/acxwigylke.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/aczrgyricp.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/adhsbajydo.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/adohikbdaz.mp4',
 '../input/deepfake-detection-challenge/train_sample_videos/adylbeequz.mp4',

In [7]:
# original이 있는 FAKE
train_sample_metadata['FAKE has original'] = [(train_sample_metadata.iloc[:,2][i] in list(train_sample_metadata.index)) for i in train_sample_metadata.T]

In [8]:
len(train_sample_metadata[train_sample_metadata['FAKE has original']]['original'])

58

In [9]:
len(train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique())

42

In [10]:
original_vidio_has_fake = train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique()
original_vidio_has_fake

array(['atvmxvwyns.mp4', 'ccfoszqabv.mp4', 'dlpoieqvfb.mp4',
       'aytzyidmgs.mp4', 'dkuayagnmc.mp4', 'bzythlfnhq.mp4',
       'cppdvdejkc.mp4', 'bulkxhhknf.mp4', 'edyncaijwx.mp4',
       'dzyuwjkjui.mp4', 'efwfxwwlbw.mp4', 'cprhtltsjp.mp4',
       'djxdyjopjd.mp4', 'cpjxareypw.mp4', 'dbtbbhakdv.mp4',
       'ellavthztb.mp4', 'bxzakyopjf.mp4', 'cizlkenljw.mp4',
       'brwrlczjvi.mp4', 'drcyabprvt.mp4', 'crezycjqyk.mp4',
       'cyxlcuyznd.mp4', 'ekcrtigpab.mp4', 'ehtdtkmmli.mp4',
       'bwipwzzxxu.mp4', 'atkdltyyen.mp4', 'chtapglbcj.mp4',
       'caifxvsozs.mp4', 'bffwsjxghk.mp4', 'duycddgtrl.mp4',
       'ehccixxzoe.mp4', 'eckvhdusax.mp4', 'dakiztgtnw.mp4',
       'bgwmmujlmc.mp4', 'abarnvbtwb.mp4', 'bejhvclboh.mp4',
       'avmjormvsx.mp4', 'egghxjjmfg.mp4', 'dbnygxtwek.mp4',
       'cmbzllswnl.mp4', 'afoovlsmtx.mp4', 'bdnaqemxmr.mp4'], dtype=object)

In [11]:
original_vidio_has_fake_path = '../input/deepfake-detection-challenge/train_sample_videos/' + original_vidio_has_fake

In [12]:
train_sample_metadata[train_sample_metadata['original']==original_vidio_has_fake[0]].index

Index(['abofeumbvv.mp4', 'bqkdbcqjvb.mp4', 'cdyakrxkia.mp4', 'cycacemkmt.mp4',
       'czmqpxrqoh.mp4', 'dakqwktlbi.mp4'],
      dtype='object')

In [13]:
train_video_files = sorted(original_vidio_has_fake)
detector = dlib.get_frontal_face_detector()
vid_num = 0
all_vid = len(train_video_files)
train_video_files

['abarnvbtwb.mp4',
 'afoovlsmtx.mp4',
 'atkdltyyen.mp4',
 'atvmxvwyns.mp4',
 'avmjormvsx.mp4',
 'aytzyidmgs.mp4',
 'bdnaqemxmr.mp4',
 'bejhvclboh.mp4',
 'bffwsjxghk.mp4',
 'bgwmmujlmc.mp4',
 'brwrlczjvi.mp4',
 'bulkxhhknf.mp4',
 'bwipwzzxxu.mp4',
 'bxzakyopjf.mp4',
 'bzythlfnhq.mp4',
 'caifxvsozs.mp4',
 'ccfoszqabv.mp4',
 'chtapglbcj.mp4',
 'cizlkenljw.mp4',
 'cmbzllswnl.mp4',
 'cpjxareypw.mp4',
 'cppdvdejkc.mp4',
 'cprhtltsjp.mp4',
 'crezycjqyk.mp4',
 'cyxlcuyznd.mp4',
 'dakiztgtnw.mp4',
 'dbnygxtwek.mp4',
 'dbtbbhakdv.mp4',
 'djxdyjopjd.mp4',
 'dkuayagnmc.mp4',
 'dlpoieqvfb.mp4',
 'drcyabprvt.mp4',
 'duycddgtrl.mp4',
 'dzyuwjkjui.mp4',
 'eckvhdusax.mp4',
 'edyncaijwx.mp4',
 'efwfxwwlbw.mp4',
 'egghxjjmfg.mp4',
 'ehccixxzoe.mp4',
 'ehtdtkmmli.mp4',
 'ekcrtigpab.mp4',
 'ellavthztb.mp4']

In [14]:
try:
    os.makedirs('../working/dataset/FAKE')
except:
    pass
try:
    os.makedirs('../working/dataset/REAL')
except:
    pass

In [15]:
try:
    os.makedirs('../working/dataset/FAKE2')
except:
    pass
try:
    os.makedirs('../working/dataset/REAL2')
except:
    pass

In [16]:
for vid in train_video_files:
    count = 0
    
    path = '../input/deepfake-detection-challenge/train_sample_videos/' + vid
    file_name = vid.split('.')[0]
    face_coord = {}
    cap = cv2.VideoCapture(path)
    frame = 0
    while(cap.isOpened()):
        ret = cap.grab()
        if ret == False:
            break
        if frame % 30 == 0:
            face_coord[frame] = []
            
            ret, image = cap.retrieve()
            face_rects, scores, idx = detector.run(image, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                li = [x1,y1,x2,y2]
                li = [0 if i<0 else li[idx] for idx, i in enumerate(li)]
                x1,y1,x2,y2 = li
                

                face_coord[frame].append(li)
                
                crop_img = image[y1:y2, x1:x2]
                landmarks = face_recognition.face_landmarks(crop_img)
                pil = Image.fromarray(crop_img)
#                 p = ImageDraw.Draw(pil)
#                 for idx, landmark in enumerate(landmarks):
#                     for mark in landmark.keys():
#                         p.line(landmark[mark], width=3)
                        
                resize_img = cv2.resize(np.array(pil), (128, 128))
                
                num = 0
                for fake_vid in train_sample_metadata[train_sample_metadata['original']==vid].index:
                    cv2.imwrite(f'../working/dataset/REAL2/{file_name}_{count}_REAL_NONE_{num}.jpg', resize_img)
                    num += 1
                print(f'../working/dataset/REAL2/{file_name}_{count}_REAL_NONE.jpg')
                count += 1
        frame += 1
    print(f'{vid_num}/{all_vid}')
    vid_num += 1
    cap.release()
    
    
    # 이비디오의 fake 영상만큼 반복
    fake_num = 0
    for fake_vid in train_sample_metadata[train_sample_metadata['original']==vid].index:
        count2 = 0
        fake_path = '../input/deepfake-detection-challenge/train_sample_videos/' + fake_vid
        cap2 = cv2.VideoCapture(fake_path)
        fake_vid_name = fake_vid.split('.')[0]        
        frame2 = 0
        while(cap2.isOpened()):
            ret2 = cap2.grab()
            if ret2 == False:
                break
            if frame2 % 30 == 0:
                ret2, image2 = cap2.retrieve()
                for i in face_coord[frame2]:
                    x1,y1,x2,y2 = i
                    crop_img2 = image2[y1:y2, x1:x2]
                    landmarks2 = face_recognition.face_landmarks(crop_img2)
                    pil2 = Image.fromarray(crop_img2)
#                     p2 = ImageDraw.Draw(pil2)
#                     for idx2, landmark2 in enumerate(landmarks2):
#                         for mark2 in landmark2.keys():
#                             p2.line(landmark2[mark2], width=3)
                    cv2.imwrite(f'../working/dataset/FAKE2/{fake_vid_name}_{count}_FAKE_{file_name}_{fake_num}.jpg', cv2.resize(np.array(pil2), (128, 128)))
                    print(f'../working/dataset/FAKE2/{fake_vid_name}_{count}_FAKE_{file_name}_{fake_num}.jpg')
                    fake_num += 1
            frame2 += 1
        cap2.release()
        count2 += 1

../working/dataset/REAL2/abarnvbtwb_0_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_1_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_2_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_3_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_4_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_5_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_6_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_7_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_8_REAL_NONE.jpg
../working/dataset/REAL2/abarnvbtwb_9_REAL_NONE.jpg
0/42
../working/dataset/FAKE2/eepezmygaq_10_FAKE_abarnvbtwb_0.jpg
../working/dataset/FAKE2/eepezmygaq_10_FAKE_abarnvbtwb_1.jpg
../working/dataset/FAKE2/eepezmygaq_10_FAKE_abarnvbtwb_2.jpg
../working/dataset/FAKE2/eepezmygaq_10_FAKE_abarnvbtwb_3.jpg
../working/dataset/FAKE2/eepezmygaq_10_FAKE_abarnvbtwb_4.jpg
../working/dataset/FAKE2/eepezmygaq_10_FAKE_abarnvbtwb_5.jpg
../working/dataset/FAKE2/eepezmygaq_10_FAKE_abarnvbtwb_6.jpg
../working/dataset/FAKE2/eepezmygaq_10_FAKE_abar

../working/dataset/FAKE2/dfbpceeaox_10_FAKE_bffwsjxghk_3.jpg
../working/dataset/FAKE2/dfbpceeaox_10_FAKE_bffwsjxghk_4.jpg
../working/dataset/FAKE2/dfbpceeaox_10_FAKE_bffwsjxghk_5.jpg
../working/dataset/FAKE2/dfbpceeaox_10_FAKE_bffwsjxghk_6.jpg
../working/dataset/FAKE2/dfbpceeaox_10_FAKE_bffwsjxghk_7.jpg
../working/dataset/FAKE2/dfbpceeaox_10_FAKE_bffwsjxghk_8.jpg
../working/dataset/FAKE2/dfbpceeaox_10_FAKE_bffwsjxghk_9.jpg
../working/dataset/REAL2/bgwmmujlmc_0_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_1_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_2_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_3_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_4_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_5_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_6_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_7_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_8_REAL_NONE.jpg
../working/dataset/REAL2/bgwmmujlmc_9_REAL_NONE.jpg
9/42
../working/dataset/FAKE2/dzvyfiarrq_10_FAKE_bgwm

../working/dataset/FAKE2/ajwpjhrbcv_10_FAKE_bzythlfnhq_8.jpg
../working/dataset/FAKE2/ajwpjhrbcv_10_FAKE_bzythlfnhq_9.jpg
../working/dataset/REAL2/caifxvsozs_0_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_1_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_2_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_3_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_4_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_5_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_6_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_7_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_8_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_9_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_10_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_11_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_12_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_13_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_14_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_15_REAL_NONE.jpg
../working/dataset/REAL2/caifxvsozs_16_R

../working/dataset/FAKE2/alaijyygdv_7_FAKE_cppdvdejkc_5.jpg
../working/dataset/FAKE2/alaijyygdv_7_FAKE_cppdvdejkc_6.jpg
../working/dataset/FAKE2/dhevettufk_7_FAKE_cppdvdejkc_7.jpg
../working/dataset/FAKE2/dhevettufk_7_FAKE_cppdvdejkc_8.jpg
../working/dataset/FAKE2/dhevettufk_7_FAKE_cppdvdejkc_9.jpg
../working/dataset/FAKE2/dhevettufk_7_FAKE_cppdvdejkc_10.jpg
../working/dataset/FAKE2/dhevettufk_7_FAKE_cppdvdejkc_11.jpg
../working/dataset/FAKE2/dhevettufk_7_FAKE_cppdvdejkc_12.jpg
../working/dataset/FAKE2/dhevettufk_7_FAKE_cppdvdejkc_13.jpg
../working/dataset/REAL2/cprhtltsjp_0_REAL_NONE.jpg
../working/dataset/REAL2/cprhtltsjp_1_REAL_NONE.jpg
../working/dataset/REAL2/cprhtltsjp_2_REAL_NONE.jpg
../working/dataset/REAL2/cprhtltsjp_3_REAL_NONE.jpg
../working/dataset/REAL2/cprhtltsjp_4_REAL_NONE.jpg
../working/dataset/REAL2/cprhtltsjp_5_REAL_NONE.jpg
../working/dataset/REAL2/cprhtltsjp_6_REAL_NONE.jpg
../working/dataset/REAL2/cprhtltsjp_7_REAL_NONE.jpg
../working/dataset/REAL2/cprhtltsjp_8_RE

../working/dataset/FAKE2/bofqajtwve_10_FAKE_dbtbbhakdv_17.jpg
../working/dataset/FAKE2/bofqajtwve_10_FAKE_dbtbbhakdv_18.jpg
../working/dataset/FAKE2/bofqajtwve_10_FAKE_dbtbbhakdv_19.jpg
../working/dataset/REAL2/djxdyjopjd_0_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_1_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_2_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_3_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_4_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_5_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_6_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_7_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_8_REAL_NONE.jpg
../working/dataset/REAL2/djxdyjopjd_9_REAL_NONE.jpg
28/42
../working/dataset/FAKE2/azsmewqghg_10_FAKE_djxdyjopjd_0.jpg
../working/dataset/FAKE2/azsmewqghg_10_FAKE_djxdyjopjd_1.jpg
../working/dataset/FAKE2/azsmewqghg_10_FAKE_djxdyjopjd_2.jpg
../working/dataset/FAKE2/azsmewqghg_10_FAKE_djxdyjopjd_3.jpg
../working/dataset/FAKE2/azsmewqghg_10_FAKE_

../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_10.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_11.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_12.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_13.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_14.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_15.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_16.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_17.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_18.jpg
../working/dataset/FAKE2/drtbksnpol_10_FAKE_dzyuwjkjui_19.jpg
../working/dataset/FAKE2/dsdoseflas_10_FAKE_dzyuwjkjui_20.jpg
../working/dataset/FAKE2/dsdoseflas_10_FAKE_dzyuwjkjui_21.jpg
../working/dataset/FAKE2/dsdoseflas_10_FAKE_dzyuwjkjui_22.jpg
../working/dataset/FAKE2/dsdoseflas_10_FAKE_dzyuwjkjui_23.jpg
../working/dataset/FAKE2/dsdoseflas_10_FAKE_dzyuwjkjui_24.jpg
../working/dataset/FAKE2/dsdoseflas_10_FAKE_dzyuwjkjui_25.jpg
../worki

../working/dataset/FAKE2/cdphtzqrvp_10_FAKE_ehtdtkmmli_5.jpg
../working/dataset/FAKE2/cdphtzqrvp_10_FAKE_ehtdtkmmli_6.jpg
../working/dataset/FAKE2/cdphtzqrvp_10_FAKE_ehtdtkmmli_7.jpg
../working/dataset/FAKE2/cdphtzqrvp_10_FAKE_ehtdtkmmli_8.jpg
../working/dataset/FAKE2/cdphtzqrvp_10_FAKE_ehtdtkmmli_9.jpg
../working/dataset/REAL2/ekcrtigpab_0_REAL_NONE.jpg
../working/dataset/REAL2/ekcrtigpab_1_REAL_NONE.jpg
../working/dataset/REAL2/ekcrtigpab_2_REAL_NONE.jpg
../working/dataset/REAL2/ekcrtigpab_3_REAL_NONE.jpg
../working/dataset/REAL2/ekcrtigpab_4_REAL_NONE.jpg
../working/dataset/REAL2/ekcrtigpab_5_REAL_NONE.jpg
../working/dataset/REAL2/ekcrtigpab_6_REAL_NONE.jpg
40/42
../working/dataset/FAKE2/bvzjkezkms_7_FAKE_ekcrtigpab_0.jpg
../working/dataset/FAKE2/bvzjkezkms_7_FAKE_ekcrtigpab_1.jpg
../working/dataset/FAKE2/bvzjkezkms_7_FAKE_ekcrtigpab_2.jpg
../working/dataset/FAKE2/bvzjkezkms_7_FAKE_ekcrtigpab_3.jpg
../working/dataset/FAKE2/bvzjkezkms_7_FAKE_ekcrtigpab_4.jpg
../working/dataset/FAKE2/